<a href="https://colab.research.google.com/github/isaidv/AI/blob/main/News_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analysis of Financial News using H20

Resourses:

https://h2o-release.s3.amazonaws.com/h2o/rel-3.46.0/1/docs-website/h2o-docs/index.html

https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/index.html

https://github.com/Avkash/mldl/blob/master/orgs/h2o/guide/algo/w2v/h2o-w2v-craigslistjobs-python.md


https://gist.github.com/bhaettasch/d7f4e22e79df3c8b6c20?permalink_comment_id=3162425

https://github.com/h2oai/h2o-meetups/blob/master/2017_11_14_NLP_H2O/Amazon%20Reviews.ipynb

In [1]:
!pip install unidecode gensim -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.8 MB/s eta 0:00:00


In [2]:
#!pip install yfinance --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.37
    Uninstalling yfinance-0.2.37:
      Successfully uninstalled yfinance-0.2.37


In [3]:
# to read and manipulate the data
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', None)    # setting column to the maximum column width as per the data

# to visualise data
import matplotlib.pyplot as plt
import seaborn as sns

# to use regular expressions for manipulating text data
import re

# to manipulate string data
import string

# to remove the accented characters
import unidecode

# to load the natural language toolkit
import nltk
nltk.download('stopwords')    # loading the stopwords
nltk.download('wordnet')    # loading the wordnet module that is used in stemming

# to remove common stop words
from nltk.corpus import stopwords

# to visualize text data using wordcloud
# from wordcloud import STOPWORDS

# to perform stemming
from nltk.stem.porter import PorterStemmer


# Used in tokenization
from spacy.lang.en import English



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
! apt-get install default-jre
!java -version

! pip install h2o

import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator

h2o.init(max_mem_size = "10G")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jre-headless fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libxkbfile1 libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  default-jre default-jre-headless fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libxkbfile1 libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
0 upgraded, 12 newly installed, 0 to remove and 45 not upgraded.
Need to get 3,720 kB of archives.
After this operation, 12.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 default-jre-headless amd64 2:1.11-72build2 [3,042 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxtst6 amd64 2:1.2.3-1build4 [13.4 kB]
Get:3 http://archive.ubuntu.com

H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_unknownUser_c3wvjs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [9]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
w2v_frame=h2o.import_file("/content/drive/MyDrive/numberbatch-19.08.txt.gz")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
w2v_frame.head(20)

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200
9161912,300,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
/c/af/##,-0.0163,-0.0815,0.0081,0.1418,-0.0311,0.0125,-0.0345,-0.0424,0.0482,0.1254,-0.008,-0.0037,-0.0375,-0.0535,0.2288,-0.0145,-0.0628,0.0715,-0.0075,0.0173,0.1867,0.014,-0.0745,-0.1002,-0.037,0.0325,-0.0175,0.0929,0.1281,-0.071,-0.0761,-0.1076,-0.0652,-0.1149,-0.0476,-0.0688,0.0814,0.0784,0.0076,-0.0411,0.0803,0.0226,0.0044,-0.0651,0.0397,-0.0234,0.1153,-0.1055,-0.1005,-0.0483,-0.037,-0.0452,0.021,0.0713,0.1538,0.0549,-0.0579,0.0091,-0.0246,-0.0227,0.0322,-0.0931,-0.0252,-0.0671,0.0408,-0.0574,0.1668,-0.0822,0.0254,-0.0432,0.1424,0.0387,-0.073,-0.0467,-0.0708,0.0191,-0.0796,-0.0181,-0.0061,0.1162,-0.0051,0.0036,-0.1055,0.0609,-0.0208,-0.0023,0.1107,0.077,0.0284,0.0124,-0.0136,0.0334,0.0695,0.0564,-0.0494,0.0231,0.0274,-0.0558,-0.0819,-0.0325,0.0649,-0.0464,-0.0391,-0.0192,0.1009,-0.0622,0.0182,0.0272,-0.114,0.0572,0.063,0.0201,-0.0148,-0.0794,-0.0203,0.0866,-0.0125,-0.0269,-0.0091,0.0351,-0.0144,-0.0204,0.0967,0.0129,0.0691,0.0427,-0.0547,-0.0119,-0.0835,0.0384,0.0837,-0.0503,-0.0898,0.0811,-0.0167,0.0507,-0.0481,-0.0795,0.0182,0.017,0.0516,-0.05,-0.054,-0.0147,0.0248,-0.0834,-0.058,0.0837,0.0081,0.0489,-0.0358,0.0343,-0.108,0.0184,0.0062,-0.0485,0.0577,-0.044,0.0852,0.0886,-0.0971,-0.0377,-0.0865,-0.0862,-0.0327,-0.0389,0.0056,-0.0315,0.0083,-0.0027,-0.0201,0.0442,-0.0534,0.0336,0.0388,-0.0384,-0.0225,-0.0389,-0.0455,-0.1187,-0.0098,0.015,-0.0386,-0.0202,0.0665,0.1001,-0.0767,0.0078,-0.0561,0.0776,-0.0667,0.0783,0.0155,0.0036,-0.0278,-0.0132,-0.0148,-0.0863,0.0034
/c/af/###,-0.0338,-0.0758,-0.0801,0.1171,-0.0742,0.0002,0.014,0.0105,0.0157,0.0663,-0.0259,0.0355,-0.0472,-0.0636,0.2448,-0.0265,-0.0803,0.0987,-0.0166,0.042,0.1472,0.1088,-0.0952,-0.0432,-0.0107,-0.0048,-0.0103,0.0577,0.2206,-0.1128,-0.0659,-0.0839,-0.0243,-0.083,0.0019,-0.147,0.0698,0.0672,0.0173,0.0751,0.0729,-0.005,-0.0272,-0.0735,0.0577,0.045,0.0248,-0.0214,-0.0901,-0.0553,0.0138,-0.0025,-0.0123,0.1242,0.0607,0.0097,-0.0693,-0.0077,-0.0923,0.0375,0.067,-0.1034,-0.0501,-0.0833,-0.0311,-0.0125,0.0996,-0.0254,0.0256,-0.0671,0.0475,0.1071,-0.0144,-0.1197,-0.1077,-0.0045,-0.0428,0.1159,0.0277,0.0534,-0.0382,0.0799,-0.089,0.0717,-0.0364,0.0427,0.0948,0.0108,0.0023,-0.0078,0.0037,0.0228,0.0535,0.0456,-0.125,0.0326,0.0855,-0.0593,-0.038,-0.0189,0.0902,-0.0512,-0.0972,0.0341,0.1399,0.0161,0.0185,0.0392,

In [ ]:
News_to_Token = h2o.import_file("/content/drive/MyDrive/analyst_ratings_processed.csv", destination_frame = "NewstoToken")
                             #col_names = ["author", "contents", "teaser", "title"], col_types = ["string", "string", "string", "string"], header = 1)

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def tokenize(sentences, stop_word = stop_words):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(stop_words)),:]
    return tokenized_words

In [ ]:
words = tokenize(News_to_Token["contents"])

In [ ]:
w2v_model = H2OWord2vecEstimator(pre_trained = w2v_frame, vec_size = 100)
w2v_model.train(training_frame = words)